In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE107754"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE107754"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE107754.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE107754.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE107754.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# The Series_title and Series_summary mention "whole human genome gene expression microarrays"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Examining the Sample Characteristics Dictionary:

# For Trait (Cervical Cancer):
# From key 2, we can see 'tissue: Cervical cancer' and 'tissue: Cervix cancer'
trait_row = 2

# For Age:
# There's no age information in the sample characteristics dictionary
age_row = None

# For Gender:
# From key 0, we can see 'gender: Male' and 'gender: Female'
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait (cancer type) to binary (1 for cervical cancer, 0 for other cancers)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Check for cervical cancer
    if 'cervix cancer' in value or 'cervical cancer' in value:
        return 1
    else:
        return 0

# No age data, but define the function for consistency
def convert_age(value):
    """Convert age to continuous value"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical data
    try:
        # Assuming clinical_data has been defined in a previous step
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical features
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Save clinical features to CSV
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data not defined. Make sure to load clinical data in a previous step.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers are from Agilent microarray probes (the A_23_P format is characteristic of Agilent)
# They are not standard human gene symbols, but probe IDs that need to be mapped to gene symbols
# Based on my biomedical knowledge, these are microarray probe identifiers, not gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for gene identifier and gene symbol
# Based on the preview of the gene annotation data, 'ID' matches the gene expression data identifiers
# and 'GENE_SYMBOL' contains the gene symbols we need

# 2. Get the gene mapping dataframe using the function from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Print a preview of the mapping to verify
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This function handles the case where one probe maps to multiple genes
# For these cases, it splits the expression values equally among mapped genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the resulting gene expression matrix
print(f"\nGene expression matrix shape after mapping: {gene_data.shape}")
# Preview the first few genes
print("\nFirst 5 gene symbols after mapping:")
print(gene_data.index[:5])


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This is an HPV-transformed keratinocyte cell line study focusing on transformation stages: 1 for anchorage independent (more advanced cancer stage), 0 for earlier stages."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")